In [ ]:
# PARCIAL 1: Clasificación de Imágenes - Carlos Morales (2360563)

# --- Configuración inicial para Google Colab ---
# Importación de librerías necesarias para el procesamiento, entrenamiento y visualización
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
import numpy as np

# Verificar si se detecta una GPU para acelerar el entrenamiento
print("GPU disponible:", tf.config.list_physical_devices('GPU'))

# Montar Google Drive para guardar y cargar modelos
from google.colab import drive
drive.mount('/content/drive')

# --- PARTE 1: Clasificación de Imágenes de Flores (tf_flowers) ---
# Carga del conjunto de datos tf_flowers
# Este dataset contiene imágenes de diferentes tipos de flores con etiquetas correspondientes.
dataset_name = 'tf_flowers'
(raw_train, raw_validation), metadata = tfds.load(
    dataset_name,
    split=['train[:80%]', 'train[80%:]'],  # División: 80% entrenamiento, 20% validación
    with_info=True,  # Incluye metadatos
    as_supervised=True  # Retorna datos en formato (imagen, etiqueta)
)

# Visualización de imágenes
# Función que toma un conjunto de datos y muestra las primeras 9 imágenes con sus etiquetas
def visualize_data(dataset, title):
    plt.figure(figsize=(10, 10))  # Tamaño de la figura
    for i, (image, label) in enumerate(dataset.take(9)):  # Iterar sobre 9 muestras
        plt.subplot(3, 3, i + 1)  # Organizar en cuadrícula 3x3
        plt.imshow(image)  # Mostrar la imagen
        plt.title(f"Etiqueta: {label.numpy()}")  # Título con la etiqueta numérica
        plt.axis("off")  # Ocultar los ejes para claridad
    plt.suptitle(title)  # Título general
    plt.show()

# Visualización del conjunto de entrenamiento
visualize_data(raw_train, "Conjunto de Entrenamiento - tf_flowers")

# Preprocesamiento de imágenes
# Función que redimensiona las imágenes a 128x128 píxeles y normaliza los valores a [0, 1]
IMG_SIZE = (128, 128)  # Tamaño deseado para las imágenes
def preprocess(image, label):
    image = tf.image.resize(image, IMG_SIZE)  # Cambiar tamaño
    image = image / 255.0  # Normalizar
    return image, label

# Preprocesar y agrupar imágenes en lotes
# `prefetch` optimiza la carga paralela de datos para el entrenamiento
train = raw_train.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
validation = raw_validation.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Construcción del modelo CNN para tf_flowers
# Este modelo utiliza varias capas convolucionales, de pooling y densas para clasificar flores
flower_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # Primera capa convolucional
    layers.MaxPooling2D((2, 2)),  # Reducción de tamaño con max pooling
    layers.Conv2D(64, (3, 3), activation='relu'),  # Segunda capa convolucional
    layers.MaxPooling2D((2, 2)),  # Segunda capa de pooling
    layers.Conv2D(128, (3, 3), activation='relu'),  # Tercera capa convolucional
    layers.MaxPooling2D((2, 2)),  # Tercera capa de pooling
    layers.Flatten(),  # Convertir las matrices 2D en un vector
    layers.Dense(128, activation='relu'),  # Capa completamente conectada
    layers.Dense(metadata.features['label'].num_classes, activation='softmax')  # Capa de salida
])

# Compilación del modelo
# Optimizador Adam, función de pérdida para clasificación múltiple y precisión como métrica
flower_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Entrenamiento del modelo
# Se entrena durante 5 épocas y se valida en cada época
flower_history = flower_model.fit(
    train,
    validation_data=validation,
    epochs=5
)

# Evaluación del modelo
# Se evalúa el modelo en el conjunto de validación para medir su rendimiento
flower_loss, flower_accuracy = flower_model.evaluate(validation)
print(f"Modelo de Flores - Precisión en validación: {flower_accuracy * 100:.2f}%")

# Visualización de resultados
# Graficar precisión y pérdida durante el entrenamiento
def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Validación')
    plt.title(f'{title} - Precisión')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.title(f'{title} - Pérdida')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.show()

# Llama a la función para graficar resultados del modelo de flores
plot_history(flower_history, "Modelo de Flores")

# Guardar el modelo entrenado
flower_model.save('/content/drive/My Drive/flower_model.h5')

# --- PARTE 2: Clasificación de Imágenes de Perros y Gatos (cats_vs_dogs) ---
# Carga y procesamiento similar, con cambios en el modelo (clasificación binaria)
# Repite pasos de carga, preprocesamiento, construcción del modelo y evaluación

(raw_train_dogs, raw_validation_dogs), metadata_dogs = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)
train_dogs = raw_train_dogs.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dogs = raw_validation_dogs.map(preprocess).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

dogs_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Activación sigmoide para clasificación binaria
])
dogs_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
dogs_history = dogs_model.fit(train_dogs, validation_data=validation_dogs, epochs=5)
dogs_loss, dogs_accuracy = dogs_model.evaluate(validation_dogs)
print(f"Modelo de Perros y Gatos - Precisión en validación: {dogs_accuracy * 100:.2f}%")

# --- Comparación entre los modelos ---
# 1. Diferencias en las salidas: softmax para clasificación múltiple (flores) vs sigmoide (perros y gatos).
# 2. Datasets: `tf_flowers` tiene múltiples clases; `cats_vs_dogs` solo dos clases.
# 3. Resultados dependen de la complejidad de la tarea y del dataset.

# Conclusión
print("\n--- CONCLUSIÓN ---")
print("El modelo de flores obtuvo una precisión del {:.2f}% en validación.".format(flower_accuracy * 100))
print("El modelo de perros y gatos obtuvo una precisión del {:.2f}% en validación